In [8]:
import pandas as pd

In [9]:
def query_mariadb(query):
    import pandas as pd
    import pymysql
    import mariadb
 
    # Connect to MariaDB 
    dbconn = mariadb.connect(
        user="root",
        password="pass123",
        host="127.0.0.1",
        port=3307,
        database="test"
    )
    
    # mariaDB Query to Pandas DataFrame
    query_result= pd.read_sql(query,dbconn)
    dbconn.close()
 
    return query_result

In [13]:
sql = "SELECT * FROM person"
df = query_mariadb(sql)
df

,ID,Nama,email,mobile,credit
0,1,Marvin Geistmann,mgeistmann0@accuweather.co,087808243908,1234 4214 1521 5132
1,2,Hermia Chapleo,hchapleo1@macromedia.com,087808243909,1234 4214 1521 5133
2,3,Laird Vowels,lvowels2@ehow.com,087808243910,1235 4214 1521 5134
3,4,Deeanne Adelsberg,dadelsberg3@telegraph.co.uk,087808243911,1235 4214 1521 5135


In [16]:
def mask_email(Var):
    mail=Var.split("@")[0]
    n=len(mail)
    alist=list(mail)
    alist[1:int(n)-1]='*'*int(n-2)
    output="".join(alist)+"@"+Var.split("@")[1]
    return output

In [21]:
def mask_mobile(Var):
    Var=str(Var)
    n=len(Var)
    alist=list(Var)
    alist[2:int(n)-2]='*'*int(n-4)
    output="".join(alist)
    return output

In [22]:
import re
def mask_credit(Var):
    output = re.sub('\d', '*', Var[:-4]) + Var[-4:]
    return output

In [23]:
df['email'] = df.email.apply(mask_email)
df

,ID,Nama,email,mobile,credit
0,1,Marvin Geistmann,m*********0@accuweather.co,087808243908,1234 4214 1521 5132
1,2,Hermia Chapleo,h*******1@macromedia.com,087808243909,1234 4214 1521 5133
2,3,Laird Vowels,l******2@ehow.com,087808243910,1235 4214 1521 5134
3,4,Deeanne Adelsberg,d*********3@telegraph.co.uk,087808243911,1235 4214 1521 5135


In [24]:
df['mobile'] = df.mobile.apply(mask_mobile)
df

,ID,Nama,email,mobile,credit
0,1,Marvin Geistmann,m*********0@accuweather.co,08********08,1234 4214 1521 5132
1,2,Hermia Chapleo,h*******1@macromedia.com,08********09,1234 4214 1521 5133
2,3,Laird Vowels,l******2@ehow.com,08********10,1235 4214 1521 5134
3,4,Deeanne Adelsberg,d*********3@telegraph.co.uk,08********11,1235 4214 1521 5135


In [25]:
df['credit'] = df.credit.apply(mask_credit)
df

,ID,Nama,email,mobile,credit
0,1,Marvin Geistmann,m*********0@accuweather.co,08********08,**** **** **** 5132
1,2,Hermia Chapleo,h*******1@macromedia.com,08********09,**** **** **** 5133
2,3,Laird Vowels,l******2@ehow.com,08********10,**** **** **** 5134
3,4,Deeanne Adelsberg,d*********3@telegraph.co.uk,08********11,**** **** **** 5135


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      4 non-null      int64 
 1   Nama    4 non-null      object
 2   email   4 non-null      object
 3   mobile  4 non-null      object
 4   credit  4 non-null      object
dtypes: int64(1), object(4)
memory usage: 288.0+ bytes


In [27]:
import pyTigerGraph as tg

In [29]:
conn = tg.TigerGraphConnection(host="https://mytg.i.tgcloud.io", password="pass123")

In [34]:
print(conn.gsql('''
  CREATE VERTEX test(PRIMARY_ID id INT, name STRING, email STRING, mobile STRING, credit STRING) WITH primary_id_as_attribute="true"
''', options=[]))

Successfully created vertex types: [test].


In [35]:
print(conn.gsql('''CREATE GRAPH Mask(test)''', options=[]))

The graph Mask is created.


In [36]:
conn.graphname = "Mask"
secret = conn.createSecret()
token = conn.getToken(secret, setToken=True)
print(token)

('ui33kogl5qgvj8k07fmf68hopgje4m0o', 1647941468, '2022-03-22 09:31:08')


In [43]:
results=conn.gsql('''
  USE GRAPH Mask
  DROP JOB load_people
  BEGIN
  CREATE LOADING JOB load_people FOR GRAPH Mask {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX test VALUES($0, $1, $2, $3, $4) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  }
  END
''', options=[])
print(results)

Using graph 'Mask'
Successfully dropped jobs on the graph 'Mask': [load_people].
Successfully created loading jobs: [load_people].


In [40]:
df.to_csv("mask_data.csv", index=None)

In [45]:
import json
#Load the posts file with the 'load_people' job
people_file='mask_data.csv'
results=conn.uploadFile(people_file, fileTag='MyDataSource', jobName='load_people')
print(json.dumps(results,indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 5,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "test",
          "validObject": 4,
          "noIdFound": 0,
          "invalidAttribute": 1,
          "invalidAttributeLines": [
            "1:id"
          ],
          "invalidAttributeLinesData": [
            "ID,Nama,email,mobile,credit\r\n"
          ],
          "invalidVertexType": 0,
          "invalidPrimaryId": 1,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]
